<a href="https://colab.research.google.com/github/ssvadla/Demonstration/blob/main/SemiXGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Semi Supervised approach "Semi Supervised Unlabelled" extemsion without upsampling and undersampling.

Inference:  Can use different thresholds less than 0.98 as the number of images that are taken out from each section of unlabeled data are very less.

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

train = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [ ]:
Threshold= 0.98

In [ ]:
train['Target'].unique()

array(['Invalid', 'Others', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

(3476, 6383)


In [ ]:
test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')

test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
# fit model no training data
classifier = XGBClassifier()
classifier.fit(X_train, Y_train)

# make predictions for test data
y_pred = classifier.predict(x_val)
#predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy validation: %f" % (accuracy))

test_pred = classifier.predict(t_p)
#test_predictions = [round(value) for value in test_pred]
acc_test = accuracy_score(test['Target'], test_pred)
#acc_test = accuracy_score(test_pred,test['Target'])

print("Accuracy test : %f" % (acc_test ))

Accuracy validation: 0.654867
Accuracy test : 0.595010


In [ ]:

unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
unlabel.head()


,Unnamed: 0,text,Complete
0,0,No motion for rehearing filed.,1
1,1,— Relator is under accusation of killing her h...,1
2,2,From an order of the District Judge of the 39t...,1
3,3,Relator and her husband had been married for a...,1
4,4,The evidence indicates that their family relat...,1


In [ ]:
del unlabel['Complete']

In [ ]:
del unlabel['Unnamed: 0']

In [ ]:
unlabel.head()

,text
0,No motion for rehearing filed.
1,— Relator is under accusation of killing her h...
2,From an order of the District Judge of the 39t...
3,Relator and her husband had been married for a...
4,The evidence indicates that their family relat...


In [ ]:
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


In [ ]:
from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
unlabel.head()

,text
0,motion rehearing filed
1,relator accusation killing husband virgil clark
2,order district judge 39th judicial district re...
3,relator husband married number year
4,evidence indicates family relation always plea...


In [ ]:
len(unlabel)

537703

In [ ]:
unlabel_1 = unlabel.loc[:100000]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = unlabel.loc[100001:200000]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = unlabel.loc[200001:300000]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = unlabel.loc[300001:400000]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = unlabel.loc[400001:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 100001
length of unlabel_2 100000
length of unlabel_3 100000
length of unlabel_4 100000
length of unlabel_5 137702


In [ ]:
unlabel_1.head()

,text
0,motion rehearing filed
1,relator accusation killing husband virgil clark
2,order district judge 39th judicial district re...
3,relator husband married number year
4,evidence indicates family relation always plea...


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,text
0,responding conclusion regarding evidence annou...
1,conviction transporting intoxicating liquor pu...
2,bill exception case filed late
3,motion new trial overruled october 22 1927 ord...
4,thereafter december 10th extension order made ...


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,text
0,another man carrying clothes
1,officer testified appellant several item cloth...
2,investigation followed report breaking entry a...
3,appellant present
4,warrant arrest issued officer went hotel waite...


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,text
0,statute final judgment appeal court lie senten...
1,sentence must pronounced term court judgment r...
2,court sentence defendant vacation
3,court adjourned authority case ceased notice a...
4,pronouncing sentence authorized vacation autho...


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,text
0,member federal reserve bank
1,opinion state case
2,conviction violating state cigarette law punis...
3,find record bill exception bringing forward co...
4,brief appellant complains lack testimony


In [ ]:
classifier.classes_

array(['Analysis', 'Conclusion', 'Facts', 'Invalid', 'Issue',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['text'])


In [ ]:
x_un1.shape
pred_unlabel_1 = classifier.predict_proba(x_un1)
pred_unlabel_1



array([[0.1101056 , 0.10172036, 0.5385539 , 0.16915338, 0.03345498,
        0.04701181],
       [0.12943962, 0.03752175, 0.5294176 , 0.19885588, 0.04949832,
        0.05526686],
       [0.14074743, 0.04079964, 0.5756673 , 0.13866976, 0.04276533,
        0.06135051],
       ...,
       [0.1525092 , 0.16698521, 0.5207258 , 0.06909035, 0.05513156,
        0.03555784],
       [0.16605468, 0.03575326, 0.5044648 , 0.18948331, 0.03747581,
        0.06676812],
       [0.14216523, 0.03786865, 0.5343122 , 0.19018301, 0.03969312,
        0.05577782]], dtype=float32)

In [ ]:
print(len(pred_unlabel_1))
print(np.unique(pred_unlabel_1))


100001
[8.3677273e-04 1.1326037e-03 1.2882029e-03 ... 9.8015928e-01 9.8069888e-01
 9.8366046e-01]


In [ ]:
Threshold= 0.98

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2]
[0.98366046, 0.9806989, 0.9801593]
[19927, 36287, 79743]
<class 'list'>
3
3
3


In [ ]:
unlabel_1 = unlabel_1.loc[pos,:]
print(len(unlabel_1))
unlabel_1.head()

3


,text
19927,appellant testified bed asleep wife called tol...
36287,f machen testified inquired appellant wife hus...
79743,appellant testified occasion question daughter...


In [ ]:
class_x_un1 = tfidf_vect.transform(unlabel_1['text'])

In [ ]:
class_x_un1.shape

(3, 6383)

In [ ]:

class_pred_unlabel_1 = classifier.predict(class_x_un1)
class_pred_unlabel_1


array(['Facts', 'Facts', 'Facts'], dtype=object)

In [ ]:
print(len(class_pred_unlabel_1))

3


In [ ]:
unlabel_1['Target']=class_pred_unlabel_1
unlabel_1.head()

,text,Target
19927,appellant testified bed asleep wife called tol...,Facts
36287,f machen testified inquired appellant wife hus...,Facts
79743,appellant testified occasion question daughter...,Facts


In [ ]:
train = train.rename(columns={'Sentence':'text'})
train.head()

,Unnamed: 0,text,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Invalid
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [ ]:
frame_1 = [train,unlabel_1]
train_1 = pd.concat(frame_1)
print(len(train))
print(len(unlabel_1))
print(len(train_1))

3476
3
3479


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
# fit model no training data
classifier_1 = XGBClassifier()
classifier_1.fit(X_train, Y_train)

# make predictions for test data
y_pred = classifier_1.predict(x_val)
#predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy validation: %f" % (accuracy))

test_pred = classifier_1.predict(t_p)
#test_predictions = [round(value) for value in test_pred]
acc_test = accuracy_score(test['Target'], test_pred)
#acc_test = accuracy_score(test_pred,test['Target'])

print("Accuracy test : %f" % (acc_test ))

Accuracy validation: 0.654867
Accuracy test : 0.595010


Second set of unlabeled data 

In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['text'])


In [ ]:

pred_unlabel_2 = classifier_1.predict_proba(x_un2)
pred_unlabel_2

array([[0.15511663, 0.0425888 , 0.546506  , 0.15813883, 0.04059898,
        0.05705076],
       [0.09863124, 0.02660361, 0.23805799, 0.08848847, 0.190097  ,
        0.3581217 ],
       [0.13993214, 0.04056331, 0.49427605, 0.22296408, 0.0425176 ,
        0.05974686],
       ...,
       [0.12987107, 0.03764682, 0.5311822 , 0.19951871, 0.04633007,
        0.05545108],
       [0.13076939, 0.03790722, 0.5348564 , 0.20089878, 0.03973355,
        0.05583463],
       [0.10983369, 0.02989561, 0.7083896 , 0.06698974, 0.06280437,
        0.02208701]], dtype=float32)

In [ ]:
print(len(pred_unlabel_2))
print(np.unique(pred_unlabel_2))

100000
[5.6254666e-04 7.6142838e-04 8.7120297e-04 ... 9.8257321e-01 9.8769569e-01
 9.9075341e-01]


In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2]
[0.9876957, 0.98243284, 0.9907534, 0.9825732, 0.98117834]
[27065, 29030, 36703, 84221, 91578]
<class 'list'>
5
5
5


In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
print(len(unlabel_2))
unlabel_2.head()

5


,text
27065,testified tiny baltrip inside heard noise star...
29030,officer gallaher testified response call went ...
36703,testified week homicide dance attended decease...
84221,also testified sure wife laid bed first night ...
91578,accomplice testified appellant came witness ho...


In [ ]:
class_x_un2 = tfidf_vect.transform(unlabel_2['text'])

In [ ]:
class_x_un1.shape

(3, 6383)

In [ ]:
class_pred_unlabel_2 = classifier_1.predict(class_x_un2)
class_pred_unlabel_2

array(['Facts', 'Facts', 'Facts', 'Facts', 'Facts'], dtype=object)

In [ ]:
print(len(class_pred_unlabel_2))

5


In [ ]:
unlabel_2['Target']=class_pred_unlabel_2
unlabel_2.head()

,text,Target
27065,testified tiny baltrip inside heard noise star...,Facts
29030,officer gallaher testified response call went ...,Facts
36703,testified week homicide dance attended decease...,Facts
84221,also testified sure wife laid bed first night ...,Facts
91578,accomplice testified appellant came witness ho...,Facts


In [ ]:
train_1.head()

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue


In [ ]:
frame_2 = [train_1,unlabel_2]
train_2 = pd.concat(frame_2)
print(len(train_1))
print(len(unlabel_2))
print(len(train_2))

3479
5
3484


In [ ]:
train_2

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue
...,...,...,...
27065,NaN,testified tiny baltrip inside heard noise star...,Facts
29030,NaN,officer gallaher testified response call went ...,Facts
36703,NaN,testified week homicide dance attended decease...,Facts
84221,NaN,also testified sure wife laid bed first night ...,Facts


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
# fit model no training data
classifier_2 = XGBClassifier()
classifier_2.fit(X_train, Y_train)

# make predictions for test data
y_pred = classifier_2.predict(x_val)
#predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy validation: %f" % (accuracy))

test_pred = classifier_2.predict(t_p)
#test_predictions = [round(value) for value in test_pred]
acc_test = accuracy_score(test['Target'], test_pred)
#acc_test = accuracy_score(test_pred,test['Target'])

print("Accuracy test : %f" % (acc_test ))

Accuracy validation: 0.654867
Accuracy test : 0.595010


Unlabelled 3:

In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['text'])


In [ ]:

pred_unlabel_3 = classifier_2.predict_proba(x_un3)
pred_unlabel_3

array([[0.13076939, 0.03790722, 0.5348564 , 0.20089878, 0.03973355,
        0.05583463],
       [0.04131139, 0.01285888, 0.87644374, 0.02881403, 0.02574264,
        0.01482934],
       [0.12308408, 0.03350222, 0.67133266, 0.07507142, 0.05990423,
        0.03710537],
       ...,
       [0.10099844, 0.02749075, 0.6326032 , 0.09788227, 0.10468129,
        0.03634401],
       [0.10137656, 0.03884389, 0.62541753, 0.15925969, 0.05358655,
        0.02151573],
       [0.10174616, 0.02949401, 0.41614923, 0.37825298, 0.030915  ,
        0.04344258]], dtype=float32)

In [ ]:
print(len(pred_unlabel_2))
print(np.unique(pred_unlabel_2))

100000
[5.6254666e-04 7.6142838e-04 8.7120297e-04 ... 9.8257321e-01 9.8769569e-01
 9.9075341e-01]


In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2]
[0.9836662, 0.9809724, 0.98033005, 0.9803767]
[53373, 53471, 54031, 68064]
<class 'list'>
4
4
4


In [ ]:
unlabel_3 = unlabel_3.loc[pos,:]
print(len(unlabel_3))
unlabel_3.head()

4


,text
53373,sonny dobbin accomplice witness testified stat...
53471,prosecutrix testified substance follows time a...
54031,reference time alva hill came home testified s...
68064,appellant father testified occasion stated app...


In [ ]:
class_x_un3 = tfidf_vect.transform(unlabel_3['text'])

In [ ]:
class_x_un3.shape

(4, 6383)

In [ ]:
class_pred_unlabel_3 = classifier_2.predict(class_x_un3)
class_pred_unlabel_3

array(['Facts', 'Facts', 'Facts', 'Facts'], dtype=object)

In [ ]:
print(len(class_pred_unlabel_3))

4


In [ ]:
unlabel_3['Target']=class_pred_unlabel_3
unlabel_3.head()

,text,Target
53373,sonny dobbin accomplice witness testified stat...,Facts
53471,prosecutrix testified substance follows time a...,Facts
54031,reference time alva hill came home testified s...,Facts
68064,appellant father testified occasion stated app...,Facts


In [ ]:
train_2.head()

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue


In [ ]:
frame_3 = [train_2,unlabel_3]
train_3 = pd.concat(frame_3)
print(len(train_2))
print(len(unlabel_3))
print(len(train_3))

3484
4
3488


In [ ]:
train_3

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue
...,...,...,...
91578,NaN,accomplice testified appellant came witness ho...,Facts
53373,NaN,sonny dobbin accomplice witness testified stat...,Facts
53471,NaN,prosecutrix testified substance follows time a...,Facts
54031,NaN,reference time alva hill came home testified s...,Facts


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
# fit model no training data
classifier_3 = XGBClassifier()
classifier_3.fit(X_train, Y_train)

# make predictions for test data
y_pred = classifier_3.predict(x_val)
#predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy validation: %f" % (accuracy))

test_pred = classifier_3.predict(t_p)
#test_predictions = [round(value) for value in test_pred]
acc_test = accuracy_score(test['Target'], test_pred)
#acc_test = accuracy_score(test_pred,test['Target'])

print("Accuracy test : %f" % (acc_test ))

Accuracy validation: 0.654867
Accuracy test : 0.595010


Unlabeled 4

In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['text'])


In [ ]:

pred_unlabel_4 = classifier_3.predict_proba(x_un4)
pred_unlabel_4

array([[0.19995588, 0.05703311, 0.36015844, 0.24501233, 0.0527716 ,
        0.08506862],
       [0.13624877, 0.05267985, 0.44378445, 0.20931666, 0.04139842,
        0.11657183],
       [0.12913531, 0.04992948, 0.52817285, 0.19838837, 0.03923704,
        0.05513693],
       ...,
       [0.12450588, 0.03609157, 0.58677644, 0.1616354 , 0.03783042,
        0.0531603 ],
       [0.21351801, 0.02967633, 0.520866  , 0.15958723, 0.03261179,
        0.04374063],
       [0.05703691, 0.6048075 , 0.20884736, 0.08762483, 0.01733035,
        0.02435306]], dtype=float32)

In [ ]:
print(len(pred_unlabel_2))
print(np.unique(pred_unlabel_2))

100000
[5.6254666e-04 7.6142838e-04 8.7120297e-04 ... 9.8257321e-01 9.8769569e-01
 9.9075341e-01]


In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2]
[0.98075473, 0.9815938, 0.98121244, 0.98195136]
[18394, 45892, 78929, 84933]
<class 'list'>
4
4
4


In [ ]:
unlabel_4 = unlabel_4.loc[pos,:]
print(len(unlabel_4))
unlabel_4.head()

4


,text
18394,appellant testifying behalf stated morning jul...
45892,mr griffin checker walker grocery store lufkin...
78929,appellant testified 3rd march injured hand tre...
84933,mr swafford owner operator cafe highway 80 tes...


In [ ]:
class_x_un4 = tfidf_vect.transform(unlabel_4['text'])

In [ ]:
class_x_un4.shape

(4, 6383)

In [ ]:
class_pred_unlabel_4 = classifier_3.predict(class_x_un4)
class_pred_unlabel_4

array(['Facts', 'Facts', 'Facts', 'Facts'], dtype=object)

In [ ]:
print(len(class_pred_unlabel_4))

4


In [ ]:
unlabel_4['Target']=class_pred_unlabel_4
unlabel_4.head()

,text,Target
18394,appellant testifying behalf stated morning jul...,Facts
45892,mr griffin checker walker grocery store lufkin...,Facts
78929,appellant testified 3rd march injured hand tre...,Facts
84933,mr swafford owner operator cafe highway 80 tes...,Facts


In [ ]:
train_3.head()

,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue


In [ ]:
frame_4 = [train_3,unlabel_4]
train_4 = pd.concat(frame_4)
print(len(train_3))
print(len(unlabel_4))
print(len(train_4))
train_4

3488
4
3492


,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue
...,...,...,...
68064,NaN,appellant father testified occasion stated app...,Facts
18394,NaN,appellant testifying behalf stated morning jul...,Facts
45892,NaN,mr griffin checker walker grocery store lufkin...,Facts
78929,NaN,appellant testified 3rd march injured hand tre...,Facts


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
# fit model no training data
classifier_4 = XGBClassifier()
classifier_4.fit(X_train, Y_train)

# make predictions for test data
y_pred = classifier_4.predict(x_val)
#predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy validation: %f" % (accuracy))

test_pred = classifier_4.predict(t_p)
#test_predictions = [round(value) for value in test_pred]
acc_test = accuracy_score(test['Target'], test_pred)
#acc_test = accuracy_score(test_pred,test['Target'])

print("Accuracy test : %f" % (acc_test ))

Accuracy validation: 0.654867
Accuracy test : 0.595010


Unlabeld 5

In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['text'])


In [ ]:
x_un5.shape

(137702, 6383)

In [ ]:
pred_unlabel_5 = classifier_4.predict_proba(x_un5)
pred_unlabel_5

array([[0.13076939, 0.03790722, 0.5348564 , 0.20089878, 0.03973355,
        0.05583463],
       [0.08038323, 0.33796042, 0.21076816, 0.3030258 , 0.024424  ,
        0.04343839],
       [0.09303828, 0.02797855, 0.2371108 , 0.10437772, 0.185338  ,
        0.35215667],
       ...,
       [0.2340709 , 0.02760853, 0.40840152, 0.25888675, 0.03033944,
        0.04069284],
       [0.19517358, 0.19896252, 0.3902443 , 0.08232102, 0.07717779,
        0.05612074],
       [0.25440097, 0.16811945, 0.32413644, 0.17167626, 0.03395395,
        0.04771299]], dtype=float32)

In [ ]:
print(len(pred_unlabel_5))
print(np.unique(pred_unlabel_5)) 

137702
[0.00115497 0.0012882  0.00130931 ... 0.98153013 0.9818283  0.9838569 ]


In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> Threshold:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(type(pos))
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2]
[0.9802766, 0.9818283, 0.98063284, 0.98086435, 0.98153013, 0.9838569]
[15205, 31043, 54869, 82384, 103955, 108270]
<class 'list'>
6
6
6


In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]
print(len(unlabel_5))
unlabel_5.head()

6


,text
15205,mattie chamber husband operated past time club...
31043,appellant testified time occurrence wife separ...
54869,bass testified scared appellant companion took...
82384,joe bowe testified many year ago appellant cam...
103955,witness behalf appellant testified night quest...


In [ ]:
class_x_un5 = tfidf_vect.transform(unlabel_5['text'])

In [ ]:
class_x_un5.shape

(6, 6383)

In [ ]:
class_pred_unlabel_5 = classifier_4.predict(class_x_un5)
class_pred_unlabel_5

array(['Facts', 'Facts', 'Facts', 'Facts', 'Facts', 'Facts'], dtype=object)

In [ ]:
print(len(class_pred_unlabel_5))

6


In [ ]:
unlabel_5['Target']=class_pred_unlabel_5
unlabel_5.head()

,text,Target
15205,mattie chamber husband operated past time club...,Facts
31043,appellant testified time occurrence wife separ...,Facts
54869,bass testified scared appellant companion took...,Facts
82384,joe bowe testified many year ago appellant cam...,Facts
103955,witness behalf appellant testified night quest...,Facts


In [ ]:
train_4.head()


,Unnamed: 0,text,Target
0,659.0,Appellant had stated to the officers that she ...,Invalid
1,3456.0,We shall discuss the facts more fully in conne...,Invalid
2,2043.0,"â€œPerjury is a false statement, either writte...",Invalid
3,3344.0,The offense is felony theft by false pretext; ...,Issue
4,3231.0,Numerous contentions urging the commission of ...,Issue


In [ ]:
print(len(train_4))

3492


In [ ]:
frame_5 = [train_4,unlabel_5]
train_5 = pd.concat(frame_5)
print(len(train_4))
print(len(unlabel_5))
print(len(train_5))

3492
6
3498


In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, x_val, Y_train, y_val = train_test_split(X_tfidf,train['Target'],test_size=0.26,random_state=42)
# fit model no training data
classifier_5 = XGBClassifier()
classifier_5.fit(X_train, Y_train)

# make predictions for test data
y_pred = classifier_5.predict(x_val)
#predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy validation: %f" % (accuracy))

test_pred = classifier_5.predict(t_p)
#test_predictions = [round(value) for value in test_pred]
acc_test = accuracy_score(test['Target'], test_pred)
#acc_test = accuracy_score(test_pred,test['Target'])

print("Accuracy test : %f" % (acc_test ))

Accuracy validation: 0.654867
Accuracy test : 0.595010
